In [ ]:
# !conda create -n dlt python=3.10

In [10]:
!pip install dlt[lancedb]==0.5.1a0
!pip install sentence-transformers
!pip install ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [2]:
!yes | dlt init rest_api lancedb

Looking up the init scripts in https://github.com/dlt-hub/verified-sources.git...
Cloning and configuring a verified source rest_api (Generic API Source)
Do you want to proceed? [Y/n]: 
Verified source rest_api was added to your project!
* See the usage examples and code snippets to copy from rest_api_pipeline.py
* Add credentials for lancedb and other secrets in ./.dlt/secrets.toml
* requirements.txt was created. Install it with:
pip3 install -r requirements.txt
* Read https://dlthub.com/docs/walkthroughs/create-a-pipeline for more information
yes: стандартный вывод: Обрыв канала


In [3]:
import dlt
from dlt.destinations.adapters import lancedb_adapter
from dlt.sources.helpers.requests import Request, Response
from dlt.sources.helpers.rest_client.paginators import (
    BasePaginator,
    JSONResponsePaginator,
)
from rest_api import RESTAPIConfig, rest_api_source

In [6]:
from datetime import datetime, timezone


class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor


@dlt.resource(name="employee_handbook")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {"token": dlt.secrets["sources.rest_api.notion.api_key"]},
            "headers": {
                "Content-Type": "application/json",
                "Notion-Version": "2022-06-28",
            },
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "Homework: Employee handbook",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time",
                        },
                    },
                    "data_selector": "results",
                },
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id",
                        }
                    },
                },
            },
        ],
    }

    yield from rest_api_source(notion_config, name="employee_handbook")


def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc),
    }


@dlt.resource(
    name="employee_handbook",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time": {"dedup_sort": "desc"}},
)
def rest_api_notion_incremental(
    last_edited_time=dlt.sources.incremental(
        "last_edited_time",
        initial_value="2024-06-26T08:16:00.000Z",
        primary_key=("block_id"),
    ),
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not (len(block["content"])):
            continue
        yield block


def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(rest_api_notion_incremental, embed="content"),
        table_name="employee_handbook",
        write_disposition="merge",
    )
    print(load_info)


load_notion()

Pipeline company_policies load step completed in 0.18 seconds
1 load package(s) were loaded to destination LanceDB and into dataset notion_pages
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x7822ee2f5180> location to store data
Load package 1721658051.5742757 is LOADED and contains no failed jobs


In [7]:
import lancedb

db = lancedb.connect(".lancedb")
dbtable = db.open_table("notion_pages___employee_handbook")

dbtable.to_pandas()

,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,f389f97c-03aa-5271-b2fb-051ce16574e0,"[-0.024265619, 0.04746082, -0.011796436, 0.063...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-22 14:17:05.737345+00:00,1721657822.9845579,nRXcxezBvfC8aA
1,a9de527a-f0e9-50e9-9817-2df0ef208ba5,"[-0.04966163, 0.10853516, -0.009762589, -0.036...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-22 14:17:05.739463+00:00,1721657822.9845579,YtiggCeBKTl0lQ
2,72ac35bc-9e93-56cc-a50d-d5ebe3232d7d,"[-0.06316319, 0.17331506, 0.025351718, -0.0191...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.739574+00:00,1721657822.9845579,xEBI0qeiD4MojA
3,c595a6a7-40fe-5ce4-8a3c-37840f8c2f97,"[-0.10974315, 0.10586077, 0.0032906013, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.739674+00:00,1721657822.9845579,yleBhje56YRgRA
4,423f75c2-d53b-5d21-8e71-44f66bb4fc52,"[0.052423332, -0.06457594, 0.065862976, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.739775+00:00,1721657822.9845579,rH6WfyvMgcUBow
5,5b0983ba-a662-504d-959b-019c8ecc19a8,"[0.0005233448, -0.054883398, 0.043573365, -0.0...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.739877+00:00,1721657822.9845579,nltpwZgomTnQJg
6,9c6129ca-8c44-5eef-b513-1159281b35ef,"[0.03802629, -0.021509668, 0.047527827, 0.0647...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.739979+00:00,1721657822.9845579,jEmElvwuBHpOpw
7,18049710-1a19-5b38-a44f-b5e14761cc04,"[-0.058588073, -0.07540443, 0.033775173, 0.009...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.740082+00:00,1721657822.9845579,WZ6WMKhgF2ZluQ
8,04577168-5479-5fd0-9534-85a151fd0645,"[-0.01359925, 0.004753045, 0.024835167, 0.0159...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-22 14:17:05.740188+00:00,1721657822.9845579,cEcRZIIONXOnTg
9,d7225a37-08ef-528a-b8ee-a2d6c6a86e3b,"[0.03206087, 0.02424462, 0.00847135, 0.0317907...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-22 14:17:05.740282+00:00,1721657822.9845579,tF3B4WTj3lfZaQ


In [8]:
len(dbtable)

17

In [9]:
max(dbtable.to_pandas()["last_edited_time"])

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

In [11]:
import ollama

In [12]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):
    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [13]:
def main():
    # Connect to the lancedb table
    db = lancedb.connect(".lancedb")
    dbtable = db.open_table("notion_pages___employee_handbook")

    # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational.",
        }
    ]

    while True:
        # Accept user question
        question = input("You: ")

        # Retrieve the relevant paragraphs on the question
        context = retrieve_context_from_lancedb(dbtable, question, top_k=2)

        # Create a user prompt using the question and retrieved context
        messages.append(
            {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
        )

        # Get the response from the LLM
        response = ollama.chat(model="llama2-uncensored", messages=messages)
        response_content = response["message"]["content"]
        print(f"Assistant: {response_content}")

        # Add the response into the context window
        messages.append({"role": "assistant", "content": response_content})

In [16]:
main()

Assistant: Based on the provided context, it looks like employees are entitled to 30 days of PTO per year. If an employee has any questions or needs clarification about their PTO entitlements, they can reach out to HR for more information.
Assistant: Based on the provided context, it looks like visitors are required to sign in and show identification before entering the office premises. Employees should also keep their visitors away from areas where there are dangerous machines or sensitive equipment. Additionally, employees are encouraged to invest in their professional development by spending up to $1000 annually on educational activities or material.
Assistant: Based on the provided context, employees must ask for permission from their HR Manager before inviting visitors to the office premises. Employees should also keep their visitors away from dangerous machines, chemicals, confidential records or sensitive equipment. Additionally, employees are encouraged to invest in their profe

KeyboardInterrupt: 